# Convolutions neural nets from scratch

The goal of this document is to document my attempt to build and apply neural networks to DNA sequence data, specifically to sgRNA screening data.  The motivation of this document is Andrej Karpathy's Hacker's guide to Neural Networks. Like his work we will build our models up, starting from a one layer network on up.  The hope is that I can gain a better understanding of building deep networks and backpropagation by working my way up.  

In our data we have a 40 base pair sequence (20 bp of the sgRNA + 3 bp PAM + 10 bp upstream + 10 bp downstream - 2 bp (NGG PAM) - 1 bp from G in first position).  We'll pass each of the 4 bases in each position to a single neuron (40 in the first layer).  For each neuron $i$ in the first layer, we'll compute a score $s_{i} = a_{i} 1(A) + c_{i} 1(C) + g_{i} 1(G) + t_{i} 1(T)$ and then apply ReLU transformation (relu(x) = $\max(0, x)$).  The transformed score is then passed to a regression model, i.e. 
$$
\hat{y}_{i} = \sum_{i = 1}^{40} \beta_{i} \text{relu}(s_{i}).
$$
We'll use square loss from the observed data to train the model.

In [1]:
import numpy

def relu(x):
    return max(0.0, x)

def square_loss(y, y_hat):
    return sum((y - y_hat)*(y - y_hat))

class neuron:
    def __init__():
        # set initial values to random standard normal variates
        self.a = numpy.random.normal(0, 1, 1)
        self.c = numpy.random.normal(0, 1, 1)
        self.g = numpy.random.normal(0, 1, 1)
        self.t = numpy.random.normal(0, 1, 1)
    def __init__(a, c, g, t):
        self.a = a
        self.c = c
        self.g = g
        self.t = t        
    def compute_score(A, C, G, T):
        self.score = relu(self.a*A + self.c*C + self.g*G + self.t*T)
    def gradient(A, C, G, T):
        if self.score < 0:
            self.grad = [0, 0, 0, 0]
        else:
            self.grad = [A, C, G, T]
    def update_grad(epsilon, A, C, G, T):
        self.a += epsilon*grad[0]
        self.c += epsilon*grad[1]
        self.g += epsilon*grad[2]
        self.t += epsilon*grad[3]


class regression_gate:
    def __init__(n_params):
        self.n_params = n_params
        self.params = [0.0]*n_params
    def set_params(params):
        # make sure the lengths are correct
        assert len(params) == self.n_params, "wrong input length"
        self.params = params
    def compute_score(x):
        assert len(x) == self.n_params, "wrong input length"
        ret = 0.0
        for i in range(self.n_params):
            ret += self.params[i]*x[i]
        return ret
    def gradient(x):
        return x


        
    
    

    
    
    